In [2]:
import pandas as pd
import numpy as nm 

# For Data Encoding
from sklearn.preprocessing import MultiLabelBinarizer

# For data splitting and model evaluation
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Import the SVM classifier
from sklearn.svm import SVC

# For scaling (optional)
from sklearn.preprocessing import StandardScaler

In [3]:
# Load the dataset and Split each sequence into a list of individual emoticons
emoticon_data = pd.read_csv("/home/belief/Desktop/MLProj1/mini-project-1/datasets/train/train_emoticon.csv")
emoticon_data['split_emoticons'] = emoticon_data['input_emoticon'].apply(list)

# Remove the 'input_emoticon' column
emoticon_data = emoticon_data.drop('input_emoticon', axis=1)

print(emoticon_data)
df = emoticon_data

      label                          split_emoticons
0         0  [😛, 🛐, 😻, 😑, 😣, 🙠, 🙯, 🚼, 😒, 🙼, 😑, 🙯, 😣]
1         0  [🛐, 😑, 😪, 😛, 🚼, 🙯, 😣, 🚅, 😑, 🙯, 😹, 😣, 🙼]
2         0  [😛, 🙯, 😑, 🚡, 😣, 🚼, 🛐, 🙲, 😣, 🙯, 🛑, 😑, 🙼]
3         1  [😛, 🚼, 🛐, 🙐, 😣, 🙯, 😑, 🙪, 😑, 🙼, 🛆, 😣, 🙯]
4         1  [🛐, 🚟, 🚼, 😛, 🙋, 😑, 😣, 🙯, 😹, 🙯, 😑, 😣, 🙼]
...     ...                                      ...
7075      1  [🙯, 😺, 😻, 😑, 😣, 🛐, 😛, 🚼, 🙼, 🙯, 😣, 😑, 🚠]
7076      1  [🛐, 🚼, 😅, 🙯, 😣, 🙹, 😛, 😑, 😑, 🙼, 😣, 🚍, 🙯]
7077      1  [🛜, 😑, 🙒, 😛, 😣, 🙯, 🛐, 🚼, 🙼, 🙬, 🙯, 😣, 😑]
7078      0  [😣, 😑, 😛, 🛋, 🛐, 🙯, 🚼, 🚪, 🙯, 😑, 🛀, 😣, 🙼]
7079      0  [😛, 😣, 🚼, 😑, 🛐, 😹, 🙯, 🚭, 😑, 😣, 🚟, 🙯, 🙼]

[7080 rows x 2 columns]


In [4]:
mlb = MultiLabelBinarizer()

# Fit the binarizer and transform the split_emoticons column
emoji_encoded = mlb.fit_transform(df['split_emoticons'])

# Create a DataFrame with the encoded emojis
emoji_df = pd.DataFrame(emoji_encoded, columns=mlb.classes_)

# Concatenate the label column with the encoded emojis
final_df = pd.concat([emoji_df, df['label']], axis=1)

print(final_df.head)
print(final_df.shape)

<bound method NDFrame.head of       😀  😁  😂  😃  😄  😅  😆  😇  😉  😊  ...  🛗  🛘  🛙  🛛  🛜  🛝  🛞  🛟  🛡  label
0     0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      0
1     0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      0
2     0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      0
3     0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      1
4     0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      1
...  .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. ..    ...
7075  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      1
7076  0  0  0  0  0  1  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      1
7077  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  1  0  0  0  0      1
7078  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      0
7079  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0      0

[7080 rows x 215 columns]>
(7080, 215)


In [5]:
# Separate features (X) and labels (y)
X = final_df.drop('label', axis=1)
y = final_df['label']

In [6]:
# Split the data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [9]:
# Initialize the SVM classifier with RBF kernel
svm_classifier = SVC()

# Train the classifier
svm_classifier.fit(X_train, y_train)

SVC()

In [10]:
# Predict on the test data
y_pred = svm_classifier.predict(X_test)

In [11]:
# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.53      0.51       715
           1       0.48      0.44      0.46       701

    accuracy                           0.48      1416
   macro avg       0.48      0.48      0.48      1416
weighted avg       0.48      0.48      0.48      1416

Confusion Matrix:
[[376 339]
 [393 308]]


In [ ]:
# # Define the parameter grid
# param_grid = {
#     'C': [0.1, 1, 10, 100],        # Regularization parameter
#     'kernel': ['linear', 'rbf'],    # Kernel types
#     'gamma': ['scale', 'auto']      # Kernel coefficient
# }

# # Initialize GridSearchCV
# grid = GridSearchCV(SVC(random_state=42), param_grid, refit=True, verbose=2, cv=5)

# # Fit the grid search to the data (use scaled or unscaled data based on earlier decision)
# grid.fit(X_train, y_train)

# # Display the best parameters
# print("Best Parameters:")
# print(grid.best_params_)

# # Make predictions with the best estimator
# grid_predictions = grid.predict(X_test)

# # Evaluate the model
# print("Classification Report (Grid Search):")
# print(classification_report(y_test, grid_predictions))
